In [ ]:
import vrepimport sysimport timeimport numpy as npfrom tank import *import skfuzzy as fuzzfrom skfuzzy import control as ctrlfrom time import sleep

In [ ]:
def perpendicular_zero_stage(distance_NW_input, distance_WN_input, distance_NE_input, tank):    distance1 = ctrl.Antecedent(np.arange(0, 5, 0.01), 'distance1')    distance2 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance2')    distance3 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance3')    speed = ctrl.Consequent(np.arange(-1, 7, 0.01), 'speed')    distance1['close'] = fuzz.trapmf(distance1.universe, [0, 0, 1.5, 3])    distance1['far']   = fuzz.trapmf(distance1.universe, [1.5, 2.51, 5, 5])    distance2['close'] = fuzz.trapmf(distance2.universe, [0, 0, 1.5, 2])    distance2['far']   = fuzz.trapmf(distance2.universe, [1, 2, 4, 5])    distance3['close'] = fuzz.trapmf(distance3.universe, [0, 0, 1.5, 2])    distance3['far']   = fuzz.trapmf(distance3.universe, [1, 2, 4, 5])    speed['stop'] = fuzz.trimf(speed.universe, [-1, 0, 1])    speed['slow'] = fuzz.trimf(speed.universe, [0, 2, 4])    speed['fast'] = fuzz.trimf(speed.universe, [3, 5, 7])    rules = (        ctrl.Rule(distance1['close'] & distance2['close'] & distance3['close'], speed['fast']),        ctrl.Rule(distance1['far'] & distance1['close'] & distance3['far'], speed['stop']),        ctrl.Rule(distance1['close'] & distance2['far'], speed['fast']),        ctrl.Rule(distance1['far'] & distance2['far'], speed['fast'])    )    speed_ctrl = ctrl.ControlSystem(rules)    speed_sim = ctrl.ControlSystemSimulation(speed_ctrl)    speed_sim.input['distance2'] = distance_NW_input    speed_sim.input['distance1'] = distance_WN_input    speed_sim.input['distance3'] = distance_NE_input    speed_sim.compute()    return speed_sim.output['speed']def perpendicular_first_stage(distance_WN_input, distance_EN_input, distance_WS_input, distance_ES_input, tank):    distance1 = ctrl.Antecedent(np.arange(0, 5, 0.01), 'distance1')    distance2 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance2')    distance3 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance3')    distance4 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance4')    speed = ctrl.Consequent(np.arange(-1, 3, 0.01), 'speed')    distance1['close'] = fuzz.trapmf(distance1.universe, [0, 0, 1, 2])    distance1['far']   = fuzz.trapmf(distance1.universe, [1.5, 2.51, 5, 5])    distance2['close'] = fuzz.trapmf(distance2.universe, [0, 0, 1.5, 2])    distance2['far']   = fuzz.trapmf(distance2.universe, [1, 2, 3, 6])    distance3['close'] = fuzz.trapmf(distance3.universe, [0, 0, 1.5, 2])    distance3['far']   = fuzz.trapmf(distance3.universe, [1, 2, 3, 6])    distance4['close'] = fuzz.trapmf(distance4.universe, [0, 0, 1.5, 2])    distance4['far']   = fuzz.trapmf(distance4.universe, [1, 2, 3, 6])    speed['stop'] = fuzz.trimf(speed.universe, [-1, 0, 1])    speed['slow'] = fuzz.trimf(speed.universe, [0, 1, 2])    speed['fast'] = fuzz.trimf(speed.universe, [1, 2, 3])    rules = (        ctrl.Rule(distance1['far'] & distance2['close'], speed['fast']),        ctrl.Rule(distance1['far'] & distance2['far'],   speed['slow']),        ctrl.Rule(distance1['close'] & distance2['close'], speed['slow']),        ctrl.Rule(distance1['close'] & distance2['far'],   speed['slow']),        ctrl.Rule(distance3['far'] & distance4['close'], speed['fast']),        ctrl.Rule(distance3['far'] & distance4['far'],   speed['slow']),        ctrl.Rule(distance3['close'] & distance4['close'], speed['stop']),        ctrl.Rule(distance3['close'] & distance4['far'],   speed['slow'])    )    speed_ctrl = ctrl.ControlSystem(rules)    speed_sim = ctrl.ControlSystemSimulation(speed_ctrl)    speed_sim.input['distance1'] = distance_WN_input    speed_sim.input['distance2'] = distance_EN_input    speed_sim.input['distance3'] = distance_WS_input    speed_sim.input['distance4'] = distance_ES_input    speed_sim.compute()    return speed_sim.output['speed']def perpendicular_second_stage(distance_EN_input, distance_ES_input, distance_SE_input, distance_NE_input, tank):    distance_input = min(distance_ES_input, distance_SE_input)    distance = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance')    speed = ctrl.Consequent(np.arange(-1, 8, 0.01), 'speed')    distance['very_close'] = fuzz.trapmf(distance.universe, [0, 0, 0.79, 0.80])    distance['far'] = fuzz.trapmf(distance.universe, [0.79, 0.80, 6, 6])    speed['stop'] = fuzz.trimf(speed.universe, [-1, 0, 1])    speed['slow'] = fuzz.trimf(speed.universe, [0, 1, 2])    speed['fast'] = fuzz.trimf(speed.universe, [5, 6, 7])    rules = (        ctrl.Rule(distance['far'], speed['fast']),        ctrl.Rule(distance['very_close'], speed['stop'])    )    speed_ctrl = ctrl.ControlSystem(rules)    speed_sim = ctrl.ControlSystemSimulation(speed_ctrl)    speed_sim.input['distance'] = distance_input    speed_sim.compute()    return speed_sim.output['speed']

In [ ]:
vrep.simxFinish(-1)clientID = vrep.simxStart('127.0.0.1',19999,True,True,5000,5)if clientID == -1:    sys.exit('Could not connect')tank = Tank(clientID)

In [ ]:
proximity_sensors = ['EN','ES','NE','NW','SE','SW','WN','WS']proximity_sensors_handles = [0]*8for i in range(len(proximity_sensors)):    err_code, proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID, 'Proximity_sensor_'+proximity_sensors[i], vrep.simx_opmode_blocking)for sensor_name, sensor_handle in zip(proximity_sensors, proximity_sensors_handles):    err_code, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = vrep.simxReadProximitySensor(clientID, sensor_handle, vrep.simx_opmode_streaming)

In [ ]:
distances = {}detection_states = {}for sensor_name, sensor_handle in zip(proximity_sensors, proximity_sensors_handles):    err_code, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = vrep.simxReadProximitySensor(clientID, sensor_handle, vrep.simx_opmode_buffer)    distances[sensor_name] = np.linalg.norm(detectedPoint)    detection_states[sensor_name] = detectionStatecurrent_stage = 0t = time.time()while (time.time()-t) < 100:    for sensor_name, sensor_handle in zip(proximity_sensors, proximity_sensors_handles):        err_code, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = vrep.simxReadProximitySensor(clientID, sensor_handle, vrep.simx_opmode_buffer)        distances[sensor_name] = np.linalg.norm(detectedPoint)        detection_states[sensor_name] = detectionState    if current_stage == 0:        fuzzy_speed = perpendicular_zero_stage(distances['NW'], distances['WN'], distances['NE'], tank)        if fuzzy_speed < 3.45:            tank.forward(0)            current_stage += 1        else:            tank.forward(fuzzy_speed)    if current_stage == 1:        distances['WN'] = distances['WN'] if detection_states['WN'] else 5        distances['EN'] = distances['EN'] if detection_states['EN'] else 5        distances['WS'] = distances['WS'] if detection_states['WS'] else 5        distances['ES'] = distances['ES'] if detection_states['ES'] else 5        fuzzy_speed = perpendicular_first_stage(distances['WN'], distances['EN'], distances['WS'], distances['ES'], tank)        if fuzzy_speed < 0.51:            tank.forward(0)            current_stage += 1        else:            tank.turn_left(fuzzy_speed*1.075)            sleep(0.1)            tank.forward(fuzzy_speed*2)            sleep(0.2)    if current_stage == 2:        fuzzy_speed = perpendicular_second_stage(distances['EN'], distances['ES'], distances['SE'], distances['NE'], tank)        if fuzzy_speed < 1:            tank.forward(0)            current_stage += 1        else:            tank.leftvelocity = -fuzzy_speed            tank.rightvelocity = -fuzzy_speed/6            tank.setVelocity()